# Python Tutorial - DSO 2019 Training

This tutorial is intended to guide people wishing to use Python to participate in the challenge.

It has 5 steps:

1. Importing data
2. Descriptive analysis
3. Data Preparation
4. Creating a template
5. Calculation of predictions and submissions

# Data Import

Let's install the necessary packages for this tutorial:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
pd.set_option('display.max_columns', 500)

In [ ]:
%%time
X_train = pd.read_csv("X_train.csv", index_col=0, error_bad_lines=False)
X_test = pd.read_csv("X_test.csv", index_col=0, error_bad_lines=False)
y_train = pd.read_csv("y_train.csv", index_col=0)

In [ ]:
print("Dimension X_train:", X_train.shape)
print("Dimension X_test:", X_test.shape)

In [ ]:
X_train.head(3)

# Descriptive analysis

## Structure of the datasets

The train dataset contains the characteristics and time of sale of **8880** items sold on the Emmaus website. It is this dataset that we will use to create a model. Each object is described by an observation of X variables. These variables are described in the ```description.pdf``` file in the USB key.

The test dataset contains the characteristics of **2960 objects**, which must be predicted for the time of sale. Unlike the train, the sell time is of course not filled in and an ```id``` column has been added to identify the predictions during the submission stage.

In [ ]:
X_train.describe(include='all').T

In [ ]:
y_train.duration.value_counts()

The dataset is very balanced, each of the 3 classes has a frequency close to 1/3.

# Model Creation

Now is the time to create a model. In this tutorial we will build a Random Forest.

To do this we use the variables ```["weight","price","nb_images","image_length","image_width","category"]```.

To avoid overfitting and estimate the true performance of our model we will use the criterion of cross-validation **k-fold** method (cross-validation).

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder

### Imputation of missing values by the value "missing"

In [ ]:
X_train.category.fillna('missing', inplace=True)
X_test.category.fillna('missing', inplace=True)

### Encoding categorical features

Machine learning algorithms expect to have **numbers** as input, not strings. That's why we turn **categorical features** into numbers, using ```LabelEncoder ()```

In [ ]:
X_train.category.unique()

In [ ]:
le = LabelEncoder()
X_train['category'] = le.fit_transform(X_train.category)
X_test['category'] = le.transform(X_test.category)

In [ ]:
features = ["weight", "price", "images_count",
            "image_width", "image_height", "category"]

ppl = Pipeline([("imputer", Imputer(strategy='median')),
                ("clf", RandomForestClassifier(n_estimators=10))])

ppl.fit(X_train.loc[:, features], np.ravel(y_train))

pred_train = ppl.predict_proba(X_train.loc[:, features])
pred_cv = cross_val_predict(ppl, X_train.loc[:, features], np.ravel(y_train),
                            method='predict_proba', cv=5, n_jobs=-1)

# Calcul de l'erreur: logloss

In [ ]:
from sklearn.metrics import log_loss 

In [ ]:
print("LogLoss on train sample:",log_loss(y_pred=pred_train, y_true=y_train))
print("LogLoss on train sample (CV):",log_loss(y_pred=pred_cv, y_true=y_train))

# Calcul des predictions

In [ ]:
pred_test = ppl.predict_proba(X_test.loc[:, features])

In [ ]:
df_submission = pd.DataFrame(pred_test, index=X_test.index)

# Submission

## Possibility #1: via the QScore API

1. Go to the platform [QScore](https://qscore.datascience-olympics.com) then in "Submissions"> "Submit from your Python Notebook"
2. Get your TOKEN
3. Replace it in the function below and execute it

In [ ]:
import io, math, requests

# Only works in Python3, see comment below for Python2
def submit_prediction(df, sep=',', **kwargs):
    # TOKEN to recover on the platform: "Submissions"> "Submit from your Python Notebook"
    TOKEN='<YOUR TOKEN>'  
    URL='https://qscore.datascience-olympics.com/api/submissions'
    #buffer = io.BytesIO() # Python 2
    buffer = io.StringIO() # Python 3
    df.to_csv(buffer, sep=sep, **kwargs)
    buffer.seek(0)
    r = requests.post(URL, headers={'Authorization': 'Bearer {}'.format(TOKEN)},files={'datafile': buffer})
    if r.status_code == 429:
        raise Exception('Submissions are too close. Next submission is only allowed in {} seconds.'.format(int(math.ceil(int(r.headers['x-rate-limit-remaining']) / 1000.0))))
    if r.status_code != 200:
        raise Exception(r.text)

In [ ]:
submit_prediction(df_submission, sep=',', index=True)

## Possibility #2: Submit a CSV file

1. Go to the platform [QScore](https://qscore.datascience-olympics.com) then in "Submissions"> "Submit with a file"
2. Deposit the CSV file

In [ ]:
df_submission.to_csv("my_prediction.csv", index_label="id", header=['0', '1', '2'])